In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from datetime import datetime, timedelta

import joblib  # for saving the scaler

In [2]:
# --- 1. Read CSV of tickers and ensure correct format ---
csv_file = "environmental_ratings.csv"
tickers_df = pd.read_csv(csv_file, header=None)
tickers = tickers_df[1].unique().tolist()  # Assuming tickers are in the second column

# Convert all tickers to "TICKER-USD" format
tickers = [tk.upper() + "-USD" if not tk.endswith("-USD") else tk for tk in tickers]
tickers = tickers[1:]

print("Tickers found:", tickers)

Tickers found: ['LEO-USD', 'BUSD-USD', 'BAT-USD', 'AXS-USD', 'SAND-USD', 'QNT-USD', 'PRIME-USD', 'MANA-USD', 'GRT-USD', 'APE-USD', 'CRV-USD', 'DAI-USD', 'ONDO-USD', 'LINK-USD', 'WBTC-USD', 'POL-USD', 'SHIB-USD', 'AAVE-USD', 'BNB-USD', 'CHZ-USD', 'USDC-USD', 'UNI-USD', 'CELO-USD', 'IOTA-USD', 'CSPR-USD', 'XLM-USD', 'VET-USD', 'INJ-USD', 'BSV-USD', 'DASH-USD', 'SC-USD', 'ZK-USD', 'XTZ-USD', 'OP-USD', 'APT-USD', 'ADA-USD', 'XRP-USD', 'DOT-USD', 'ATOM-USD', 'SUI-USD', 'ALGO-USD', 'BCH-USD', 'ARB-USD', 'AVAX-USD', 'NEAR-USD', 'TRX-USD', 'TON-USD', 'ETH-USD', 'LTC-USD', 'SOL-USD', 'DOGE-USD', 'FIL-USD', 'BTC-USD']


In [3]:
# --- 2. Define date ranges ---
TRAIN_START = "2018-01-01"  # Start date for training data

# Set TRAIN_END to yesterday (or nearest previous weekday if weekend)
today = datetime.today().date()
yesterday = today - timedelta(days=1)

# Ensure yesterday is a weekday (if not, use last available trading day)
if yesterday.weekday() == 6:  # Sunday → Use Friday
    yesterday -= timedelta(days=2)
elif yesterday.weekday() == 5:  # Saturday → Use Friday
    yesterday -= timedelta(days=1)

TRAIN_END = yesterday.strftime("%Y-%m-%d")  # Convert to string format for yfinance

In [4]:
# --- 3. Fetch Data for Each Ticker ---
frames = []
failed_tickers = []

for tk in tickers:
    print(f"Fetching {tk} from {TRAIN_START} to {TRAIN_END}...")

    try:
        data = yf.download(tk, start=TRAIN_START, end=TRAIN_END)

        # Handle MultiIndex: Ensure we have a standard DataFrame
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = ['_'.join(col).strip() for col in data.columns.values]  # Flatten MultiIndex

        # Ensure the data contains "Close" column before proceeding
        if data.empty or not any(col.startswith("Close") for col in data.columns):
            print(f"Warning: No valid 'Close' column found for {tk}, skipping.")
            failed_tickers.append(tk)
            continue

        # Rename "Close" column explicitly in case of variations
        for col in data.columns:
            if col.startswith("Close"):
                data.rename(columns={col: "Close"}, inplace=True)

        # Add ticker column and store valid DataFrame
        data["Ticker"] = tk
        frames.append(data)

    except Exception as e:
        print(f"Error fetching {tk}: {e}")
        failed_tickers.append(tk)

Fetching LEO-USD from 2018-01-01 to 2025-02-28...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetching BUSD-USD from 2018-01-01 to 2025-02-28...
Fetching BAT-USD from 2018-01-01 to 2025-02-28...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetching AXS-USD from 2018-01-01 to 2025-02-28...
Fetching SAND-USD from 2018-01-01 to 2025-02-28...
Fetching QNT-USD from 2018-01-01 to 2025-02-28...


...

Fetching SOL-USD from 2018-01-01 to 2025-02-28...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetching DOGE-USD from 2018-01-01 to 2025-02-28...
Fetching FIL-USD from 2018-01-01 to 2025-02-28...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Fetching BTC-USD from 2018-01-01 to 2025-02-28...


In [5]:
# --- 4. Merge Data and Handle Missing Values ---
if frames:
    all_data = pd.concat(frames).reset_index()

    # Ensure "Close" column exists before dropping NaN values
    if "Close" in all_data.columns:
        all_data = all_data.dropna(subset=["Close"])
    else:
        print("No valid 'Close' column found in final dataset. Exiting.")
        exit(1)

    all_data.sort_values(by=["Ticker", "Date"], inplace=True)
    all_data.reset_index(drop=True, inplace=True)

    # Save Cleaned Data
    all_data.to_csv("crypto_price_data.csv", index=False)
    print(f"Data saved to crypto_price_data.csv")

else:
    print("No valid data was downloaded. Exiting.")
    exit(1)

# Print failed tickers for debugging
print(f"Failed tickers ({len(failed_tickers)}): {failed_tickers}")

Data saved to crypto_price_data.csv
Failed tickers (0): []
